<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/pubmed_twadr_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [12]:
# training data set

train_csv = path/'TwADR-L.fold-1.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [13]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-1.validation.csv",header=None)
valid.head()

,0,1
0,85,tremor
1,119,anxiety
2,154,hives
3,189,dizziness
4,189,dizziness


In [14]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-1.test.csv",header=None)
test.head()

,0,1
0,13,Insomnia
1,13,awake in the middle of the night
2,61,very stupid
3,133,night terrors
4,189,dizziness


In [15]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR_pubmed.csv')

In [16]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [17]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [18]:
data_clas.add_test(test,label=0)


In [19]:
data_lm.save()
data_clas.save()

In [20]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'diclofenac',
 'was',
 'with',
 'were',
 'for',
 'pain',
 'sodium',
 'drug',
 'by',
 'on',
 'patients',
 'as',
 'that',
 'is',
 'at',
 'or',
 'increased',
 'mg',
 'from',
 'group',
 'mood',
 'this',
 'treatment',
 'study',
 'after',
 'p',
 'release',
 'an',
 'be',
 'disorder',
 ',',
 'me',
 'abnormal',
 'i',
 'not',
 'effect',
 'withdrawal',
 'decreased',
 'weight',
 'compared',
 'than',
 'effects',
 'are',
 'anxiety',
 'groups',
 'my',
 'using',
 'symptoms',
 'no',
 'two',
 'inflammatory',
 'both',
 'significantly',
 'acute',
 'drugs',
 'significant',
 'potassium',
 'anti',
 'skin',
 'between',
 'day',
 'dose',
 'h',
 'voltaren',
 'all',
 '(',
 ')',
 'behavior',
 'disorders',
 'used',
 'blood',
 'time',
 'these',
 'results',
 'placebo',
 'it',
 'syndrome',
 '-',
 'which',
 'had',
 'efficacy',
 'diseases',
 'days',
 'sleeplessness',
 'have',
 'oral',
 'c

In [21]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [22]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.845716,4.197468,0.348738,17:09


In [24]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.021595,3.799591,0.384967,17:08


In [25]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.756328,3.754748,0.389088,17:19


In [26]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.702085,3.722140,0.392067,17:02


In [27]:
learn.save_encoder('TwADR_pubmed_fold01_first')

In [28]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR_pubmed_fold01_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.190390,6.330568,0.122807,00:25


In [29]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.199018,4.904980,0.131579,00:28


In [30]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.753134,4.636028,0.175439,01:03


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.417245,4.232177,0.219298,01:09
1,5.221195,4.121685,0.219298,01:06


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.913804,3.911737,0.228070,01:07
1,4.758351,3.812037,0.236842,01:04


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.555789,3.671383,0.263158,01:03
1,4.383978,3.538349,0.298246,01:11


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.164450,3.422315,0.315789,01:05
1,4.025812,3.313847,0.324561,01:08


In [35]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.853751,3.218666,0.342105,01:07
1,3.728204,3.183496,0.315789,01:08


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.545607,3.135198,0.342105,01:07
1,3.482176,3.037573,0.377193,01:03
2,3.302882,2.936446,0.412281,01:10
3,3.122818,2.914431,0.385965,01:03


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.919841,2.879398,0.385965,01:09
1,2.905963,2.885673,0.403509,01:04
2,2.743991,2.837632,0.421053,01:06
3,2.565166,2.806487,0.412281,01:09


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.378504,2.807933,0.429825,01:09
1,2.359831,2.883749,0.377193,01:04
2,2.190832,2.835913,0.368421,01:04
3,2.049857,2.797492,0.412281,01:09


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.913441,2.833671,0.412281,01:10
1,1.898070,2.839310,0.421053,01:09
2,1.737511,2.764351,0.403509,01:04
3,1.600939,2.751152,0.403509,01:10


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.477228,2.809476,0.438596,01:04
1,1.483428,2.829937,0.421053,01:03
2,1.371631,2.830498,0.429825,01:05
3,1.276811,2.798017,0.421053,01:08


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.201471,2.812668,0.438596,01:04
1,1.193097,3.110599,0.385965,01:04
2,1.109177,2.955318,0.429825,01:07
3,1.003279,2.893298,0.429825,01:10


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.908095,2.946764,0.421053,01:12
1,0.934451,3.058825,0.368421,01:09
2,0.862975,3.007207,0.429825,01:07
3,0.774353,2.967085,0.421053,01:07


In [43]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.739812,3.118838,0.403509,01:05
1,0.724963,3.086684,0.394737,01:08


In [44]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.691646,3.064340,0.429825,01:07
1,0.668973,3.031766,0.447368,01:04


In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.639956,3.187074,0.394737,01:06


In [46]:
# save the best model

learn.save_encoder('TwADR_pubmed_fold01')

In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.634821,3.050354,0.412281,01:09


In [48]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.585649,3.139920,0.403509,01:08


In [49]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.561617,3.225672,0.429825,01:04


# Part three: Predict on the test dataset

In [50]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [51]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

143
143
[1787, 13, 1227, 133, 1295, 189, 974, 2147, 1936, 2147, 1787, 2032, 1787, 1136, 418, 418, 1227, 744, 1227, 97, 1769, 1597, 1458, 208, 1599, 818, 2147, 825, 1484, 852, 2149, 974, 974, 792, 974, 974, 974, 978, 978, 1942, 978, 978, 1062, 109, 1669, 1089, 1096, 1787, 1175, 1682, 1936, 1936, 1706, 1295, 1526, 109, 1295, 1295, 1295, 1321, 1319, 2194, 755, 1432, 1432, 1125, 1529, 1529, 109, 233, 301, 1576, 1594, 1597, 426, 1597, 392, 392, 1610, 1610, 2194, 1526, 2149, 1227, 1073, 1432, 1940, 1531, 1531, 441, 1545, 1073, 1073, 1798, 1767, 1815, 233, 1099, 1769, 1787, 2032, 1787, 301, 2023, 923, 978, 1099, 1109, 1099, 2149, 1936, 1936, 1936, 1340, 1936, 224, 1526, 982, 1526, 1942, 441, 1813, 1718, 1787, 2149, 1787, 1340, 1340, 1974, 1529, 2081, 2147, 2147, 1099, 2147, 2032, 2147, 233, 2149, 824, 2191, 1529, 2194]
[13, 13, 61, 133, 189, 189, 195, 233, 233, 233, 393, 393, 393, 396, 418, 452, 561, 744, 749, 750, 791, 791, 809, 813, 813, 818, 824, 824, 825, 852, 947, 974, 974, 974, 974, 974

In [52]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

50
0.34965034965034963
